In [1]:
from fomo.pipelines.train import Learner
from fomo.pipelines.types.learner_args import LearnerArgs
import torch

/home/jupyter/github/fomo/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
learner_args = LearnerArgs()
learner_args.device = "cuda"
learner_args.epochs = 200
learner_args.model_type = "clip_transformer_adapter"
learner_args.train_eval_size = [288, 288]
learner_args.text_prompt_template = "a photo of a {}, a type of pet."
learner_args.print_freq = 200
learner_args.save_freq = 200
learner_args.learning_rate = 5e-3
learner_args.use_wandb = True
learner_args.patience = 20
learner_args.batch_size = 256
learner_args.momentum = 0.5
learner_args.train_subsample = "base"
learner_args.dataset = "oxford_pets"

learner = Learner(learner_args)

Turning off gradients in both the image and the text encoder
Parameters to be updated: {'adapter.mha.out_proj.weight', 'adapter.mha.in_proj_weight', 'adapter.mha.in_proj_bias', 'adapter.downsampler.1.weight', 'adapter.downsampler.1.bias', 'adapter.downsampler.0.weight', 'adapter.downsampler.0.bias', 'adapter.upsampler.0.bias', 'adapter.upsampler.2.weight', 'adapter.upsampler.2.bias', 'adapter.upsampler.0.weight', 'adapter.mha.out_proj.bias'}
Number of learnable paramms: 107712


In [3]:
learner.run()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dqmiss. Use `wandb login --relogin` to force relogin


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: [0][0/2]	Time  7.648 ( 7.648)	Data  2.500 ( 2.500)	Loss 3.4286e-01 (3.4286e-01)	Acc@1  87.11 ( 87.11)


100%|██████████| 2/2 [00:03<00:00,  1.53s/it]

Validate: [0/2]	Time  2.920 ( 2.920)	Loss 2.9327e-01 (2.9327e-01)	Prompt Acc@1  90.62 ( 90.62)
 * Prompt Acc@1 90.278


...

 * Prompt Acc@1 94.444


 12%|█▎        | 1/8 [00:03<00:27,  3.93s/it]

Validate: [0/8]	Time  3.933 ( 3.933)	Loss 1.6702e-01 (1.6702e-01)	Prompt Acc@1  96.48 ( 96.48)


100%|██████████| 8/8 [00:08<00:00,  1.12s/it]

 * Prompt Acc@1 96.343
